# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import numpy as np
import torch
from collections import deque
from time import time
import matplotlib.pyplot as plt

In [2]:
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [3]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:\n', states[0])
print('The state for the second agent looks like:\n', states[1])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
The state for the second agent looks like:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.4669857  -1.5
  0.          0.         -6.83172083  6.          0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [6]:
watch_random_agents = False
if watch_random_agents:
    for i in range(1, 6):                                      # play game for 5 episodes
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        while True:
            actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
from maddpg_agent import Group

In [8]:
group = Group(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=0)

In [9]:
def maddpg(n_episodes=2000, max_t=1000, eps_decay = 1):
    scores_deque = deque(maxlen=100)
    scores_list  = []
    solved = False
    
    for i_episode in range(1, n_episodes+1):
        t1 = time()
        
        env_info = env.reset(train_mode=True)[brain_name]
        group.reset() 
        states = env_info.vector_observations
        scores = np.zeros(num_agents) 
        
        eps = 1.0 # noise decay
        eps_min = 0.1
        
        for t in range(max_t):
            actions     = group.act(states, eps) 
            env_info    = env.step(actions)[brain_name]  # send the action to the environment
            next_states = env_info.vector_observations   # get the next state
            rewards     = env_info.rewards               # get the reward
            dones       = env_info.local_done            # see if episode has finished
            
            group.step(states, actions, rewards, next_states, dones, t) 
            
            states  = next_states
            scores += rewards
            score   = scores.max()
            
            if np.any(dones):
                break 
        
        scores_deque.append(score)
        scores_list.append(score)
        eps = max(eps_min, eps_decay*eps) # update eps
        
        t2 = time()
    
        print('\rEpisode {}\tCurrent Score {:.2f}\tAverage Score: {:.2f}\tTime: {} seconds'\
              .format(i_episode, scores_deque[-1], np.mean(scores_deque),int(t2-t1)), end='')
        
        if i_episode % 100 == 0: print()

        if np.mean(scores_deque) > 0.5 and not solved:
            print('\nProbelm solved in episode {}!'.format(i_episode))
            torch.save(group.checkpoint(), 'checkpoint_group_solved.pth')
            solved = True
    
    torch.save(group.checkpoint(), 'checkpoint_group_final.pth')
            
    return scores_list

In [ ]:
scores = maddpg(n_episodes=2000)

Episode 100	Current Score 0.00	Average Score: 0.00	Time: 0 seconds
Episode 200	Current Score 0.00	Average Score: 0.01	Time: 0 seconds
Episode 300	Current Score 0.00	Average Score: 0.01	Time: 0 seconds
Episode 400	Current Score 0.00	Average Score: 0.00	Time: 0 seconds
Episode 500	Current Score 0.00	Average Score: 0.00	Time: 0 seconds
Episode 600	Current Score 0.00	Average Score: 0.01	Time: 0 seconds
Episode 700	Current Score 0.20	Average Score: 0.07	Time: 2 seconds
Episode 800	Current Score 0.10	Average Score: 0.15	Time: 2 secondss
Episode 900	Current Score 1.20	Average Score: 0.27	Time: 14 seconds
Episode 1000	Current Score 0.10	Average Score: 0.21	Time: 0 seconds
Episode 1100	Current Score 0.20	Average Score: 0.21	Time: 2 secondss
Episode 1200	Current Score 0.10	Average Score: 0.37	Time: 2 secondss
Episode 1300	Current Score 0.30	Average Score: 0.39	Time: 7 secondss
Episode 1319	Current Score 0.40	Average Score: 0.50	Time: 6 secondss
Probelm solved in episode 1319!
Episode 1338	Curren

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()